In [1]:
import logging
import dgl
import numpy as np
from tqdm import tqdm
import torch
from pathlib import Path
import os
import sys

# sys.path.append('/zhupengfei/STExperiment/MAEST/')
# 获取当前工作目录
current_directory = os.getcwd()
print(f"当前工作目录: {current_directory}")
 
# 修改当前工作目录
new_directory = '/zhupengfei/STExperiment/MAEST/'
os.chdir(new_directory)
 
# 再次获取当前工作目录以确认更改
current_directory = os.getcwd()
print(f"新的工作目录: {current_directory}")

from utils import (
    build_args,
    create_optimizer,
    set_random_seed,
    TBLogger,
    get_current_lr,
    load_best_configs,
    build_args_known,
)
from models import build_model
from process.function import Function
import scanpy as sc
from process.preprocess import preprocess_adj, preprocess_adj_sparse, preprocess, construct_interaction, construct_interaction_KNN, get_feature
from scipy.sparse.csc import csc_matrix
from scipy.sparse.csr import csr_matrix

# 获取当前运行的脚本文件名
script_name = sys.argv[0]
 
# 获取脚本的完整路径
script_path = os.path.realpath(script_name)

script_path

/opt/conda/envs/STexp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


当前工作目录: /zhupengfei/STExperiment/MAEST/plot/Mouse_HT
新的工作目录: /zhupengfei/STExperiment/MAEST


/tmp/ipykernel_99088/2543689245.py:36: DeprecationWarning: Please use `csc_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csc` namespace is deprecated.
  from scipy.sparse.csc import csc_matrix
/tmp/ipykernel_99088/2543689245.py:37: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  from scipy.sparse.csr import csr_matrix


'/opt/conda/envs/STexp/lib/python3.10/site-packages/ipykernel_launcher.py'

In [2]:
#加载args
the_args = ['--device', '0', '--dataset', 'Mouse_HT', '--mask_method', "random", '--remask_method', "random", '--alpha_l',
             '3', '--scheduler', '--seeds', '41', '--use_cfg']

args = build_args_known(the_args)
if args.use_cfg:
    args = load_best_configs(args)
args

2024-06-10 16:25:33,869 - INFO - ----- Using best configs from configs/Mouse_HT.yaml -----


Namespace(seeds=[41], dataset='Mouse_HT', device=0, max_epoch=900, warmup_steps=-1, num_heads=8, num_out_heads=1, num_layers=1, num_dec_layers=1, num_remasking=3, num_hidden=1024, residual=False, in_drop=0.1, attn_drop=0.1, norm=None, lr=0.001, weight_decay=0.0002, negative_slope=0.2, activation='prelu', mask_rate=0.3, remask_rate=0.5, remask_method='random', mask_type='mask', mask_method='random', drop_edge_rate=0.0, encoder='gat', decoder='gat', loss_fn='sce', alpha_l=3, optimizer='adam', no_pretrain=False, load_model=False, use_cfg=True, logging=False, scheduler=True, lam=0.2, delayed_ema_epoch=0, replace_rate=0.05, momentum=0, sample=['151673'], bet=0.02, power=2, self_loop=True)

In [3]:
#基础设置
device = args.device if args.device >= 0 else "cpu"
print("所用GPU为:",device)
seed = args.seeds[0]
dataset = args.dataset
max_epoch = args.max_epoch
num_hidden = args.num_hidden
num_layers = args.num_layers
encoder_type = args.encoder
decoder_type = args.decoder
replace_rate = args.replace_rate
optim_type = args.optimizer
loss_fn = args.loss_fn
lr = args.lr
weight_decay = args.weight_decay
# save_model = args.save_model
logs = args.logging
use_scheduler = args.scheduler

dataset

所用GPU为: 0


'Mouse_HT'

In [4]:
result_path = Path('./results/' + f'{dataset}/')
# result_path.mkdir(parents=True, exist_ok=True)
#初始化模型
function = Function(datatype=dataset, n_clusters=10)
adata = function.loadData()
adata, u, v = function.process(adata)
#创建dgl数据
graph = dgl.graph((torch.tensor(u + v), torch.tensor(v + u)))
print("节点数", graph.number_of_nodes)
print("未加自环", graph.number_of_edges)
if(args.self_loop):
    graph = dgl.add_self_loop(graph)
    print("已加自环", graph.number_of_edges)
x = torch.tensor(adata.obsm['feat'])
args.num_features = x.shape[1]

节点数 <bound method DGLGraph.number_of_nodes of Graph(num_nodes=53208, num_edges=319248,
      ndata_schemes={}
      edata_schemes={})>
未加自环 <bound method DGLGraph.number_of_edges of Graph(num_nodes=53208, num_edges=319248,
      ndata_schemes={}
      edata_schemes={})>
已加自环 <bound method DGLGraph.number_of_edges of Graph(num_nodes=53208, num_edges=372456,
      ndata_schemes={}
      edata_schemes={})>


In [5]:
set_random_seed(seed)
if logs:
    logger = TBLogger(
        name=f"{dataset}_loss_{loss_fn}_rpr_{replace_rate}_nh_{num_hidden}_nl_{num_layers}_lr_{lr}_mp_{max_epoch}_mpf_{max_epoch_f}_wd_{weight_decay}_wdf_{weight_decay_f}_{encoder_type}_{decoder_type}")
else:
    logger = None

In [6]:
# 创建模型
model = build_model(args)
model.to(device)

=== Use sce_loss and alpha_l=3 ===
num_encoder_params: 3075072, num_decoder_params: 3081000, num_params_in_total: 8783842


PreModel(
  (encoder): GAT(
    (gat_layers): ModuleList(
      (0): GATConv(
        (fc): Linear(in_features=3000, out_features=1024, bias=False)
        (feat_drop): Dropout(p=0.1, inplace=False)
        (attn_drop): Dropout(p=0.1, inplace=False)
        (leaky_relu): LeakyReLU(negative_slope=0.2)
      )
    )
    (head): Identity()
  )
  (decoder): GAT(
    (gat_layers): ModuleList(
      (0): GATConv(
        (fc): Linear(in_features=1024, out_features=3000, bias=False)
        (feat_drop): Dropout(p=0.1, inplace=False)
        (attn_drop): Dropout(p=0.1, inplace=False)
        (leaky_relu): LeakyReLU(negative_slope=0.2)
      )
    )
    (head): Identity()
  )
  (encoder_to_decoder): Linear(in_features=1024, out_features=1024, bias=False)
  (projector): Sequential(
    (0): Linear(in_features=1024, out_features=256, bias=True)
    (1): PReLU(num_parameters=1)
    (2): Linear(in_features=256, out_features=1024, bias=True)
  )
  (projector_ema): Sequential(
    (0): Linear(in_feat

In [ ]:
# #加载模型
# if args.load_model:
#     model.load_state_dict(torch.load("./saved_models/MAEST_" + args.dataset + ".pt"))
#     ari=function.clusting(adata, model, graph, x, args.power, device)
#     print(ari)

In [8]:
optimizer = create_optimizer(optim_type, model, lr, weight_decay)

if use_scheduler:
    logging.info("Use schedular")
    scheduler = lambda epoch: (1 + np.cos((epoch) * np.pi / max_epoch)) * 0.5
    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=scheduler)
else:
    scheduler = None

2024-06-10 16:33:10,569 - INFO - Use schedular


In [10]:
logging.info("start training..")
graph = graph.to(args.device)
x = x.to(device)

2024-06-10 16:35:22,661 - INFO - start training..


In [11]:
# target_nodes = torch.arange(x.shape[0], device=x.device, dtype=torch.long)
epoch_iter = tqdm(range(max_epoch))
for epoch in epoch_iter:
# for epoch in range(max_epoch):
    model.train()
    loss = model(graph, x)

    loss_dict = {"loss": loss.item()}
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if scheduler is not None:
        scheduler.step()

    epoch_iter.set_description(f"# Epoch {epoch}: train_loss: {loss.item():.4f}")
    if logger is not None:
        loss_dict["lr"] = get_current_lr(optimizer)
        logger.note(loss_dict, step=epoch)

    # if (epoch + 1) % 100 == 0:
    #     #聚类
    #     function.clusting_no_label(adata, model, graph, x, power, device)

    #     # plotting spatial clustering result
    #     plt.rcParams["figure.figsize"] = (5, 5)

    #     fig, ax = plt.subplots()
    #     fig.subplots_adjust(left=0.02, right=0.82)  # 调整左边缘的边距

    #     sc.pl.embedding(adata, 
    #                     basis="spatial", 
    #                     color="domain",
    #                     s=4, 
    #                     show=False, 
    #                     title='Mouse Olf',
    #                     ax=ax
    #                     )
    #     plt.axis('off')

        # plt.savefig("./results/" + dataset + "/" + str(epoch) + ".png")
        # adata.write("./results/" + dataset + "/" + str(epoch) +".h5ad")
        # return

# Epoch 86: train_loss: 1.4680:  10%|▉         | 87/900 [01:03<09:53,  1.37it/s]


KeyboardInterrupt: 

In [ ]:
marker_genes_list=['ATP2B4', 'RASGRF2', 'LAMP5', 'NEFH', 'NTNG2', 'B3GALT2']
ax = sc.pl.stacked_violin(adata, marker_genes_list,
                          groupby='ground_truth', swap_axes=False, dendrogram=False, cmap='Reds')

In [ ]:
plot_gene = 'B3GALT2'
fig, axs = plt.subplots(1, 2, figsize=(8, 4))
sc.pl.spatial(adata, img_key="hires", color=plot_gene, show=False, ax=axs[0], title='RAW_'+plot_gene, vmax='p99')
sc.pl.spatial(adata, img_key="hires", color=plot_gene, show=False, ax=axs[1], title='STAGATE_'+plot_gene, layer='MAEST', vmax='p99')

In [ ]:
sc.pl.spatial(adata, color=marker_genes_list, ncols=2, cmap='Spectral_r')